# 将下载的压缩包统一解压

In [1]:
import zipfile
import os
import SimpleITK as sitk
import os
import pydicom
import nibabel as nib
import numpy as np
import os
import pydicom


In [1]:
# 指定要解压的ZIP文件所在的文件夹
source_folder = r"G:\CT\Fifth Hospital"

# 指定要将ZIP文件解压到的目标文件夹
destination_folder = r"G:\CT\Fifth_hospital_processed\unzip"

# 获取源文件夹中所有的ZIP文件列表
zip_files = [f for f in os.listdir(source_folder) if f.endswith('.zip')]

# 遍历每个ZIP文件并解压
for zip_file in zip_files:
    # 构建ZIP文件的完整路径
    zip_file_path = os.path.join(source_folder, zip_file)
    
    # 创建一个与ZIP文件同名的文件夹，用于解压文件
    unzip_folder = os.path.splitext(zip_file)[0]
    unzip_folder_path = os.path.join(destination_folder, unzip_folder)
    os.makedirs(unzip_folder_path, exist_ok=True)
    
    # 打开ZIP文件并解压到目标文件夹
    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        zip_ref.extractall(unzip_folder_path)

print("解压完成")


解压完成


# 基于文件夹大小来选择最高分辨率

In [3]:

def get_folder_size(folder_path):
    total_size = 0
    for dirpath, dirnames, filenames in os.walk(folder_path):
        for filename in filenames:
            file_path = os.path.join(dirpath, filename)
            total_size += os.path.getsize(file_path)
    return total_size

# 获取所有子文件夹的列表
def find_max(target_directory):

    subfolders = [f.path for f in os.scandir(target_directory) if f.is_dir()]

# 遍历子文件夹并找到占据内存最大的文件夹
    max_size = 0
    max_size_folder = None
    for folder in subfolders:
        folder_size = get_folder_size(folder)
        if folder_size > max_size:
            max_size = folder_size
            max_size_folder = folder
    return max_size_folder

def get_resolution_and_layers(filepath):
    # 初始化分辨率和层数
    resolution = None
    num_layers = 0
    
    # 列出文件夹中的所有文件
    files = os.listdir(filepath)
    
    for filename in files:
        # 拼接文件路径
        file_path = os.path.join(filepath, filename)
        if os.path.isfile(file_path):
            # 尝试读取DICOM文件
            try:
                dcm = pydicom.dcmread(file_path)
                # 检查是否是图像
                if 'PixelData' in dcm:
                    num_layers += 1
                    # 获取图像的分辨率
                    resolution = (dcm.Rows, dcm.Columns)
       
            except Exception as e:
                print(f"Error reading {file_path}: {str(e)}")
    print("File:",filepath)
    print("Resolution:", resolution)
    print("Number of layers:", num_layers) 
    return resolution, num_layers



### 读取以一个病人的为例

In [ ]:
def get_resolution_and_layers(folder_path):
    nii_files = [file for file in os.listdir(folder_path) if file.endswith('.nii.gz')]
    data_list = []

    for nii_file in nii_files:
        file_path = os.path.join(folder_path, nii_file)
        img = nib.load(file_path)
        data = img.get_fdata()
        width, height = data.shape[:2]  # 获取最后一层图层的宽度和高度
        layers = data.shape[-1]  # 获取图像总共有多少层
        resolution = img.header.get_zooms()
        
        # 将数据添加到列表中
        data_list.append([nii_file, file_path, width, height, layers, resolution])

    # 创建 DataFrame
    df = pd.DataFrame(data_list, columns=['nii_file', 'file_path', 'Width', 'Height', 'Layers', 'Resolution'])
    return df

In [2]:
test_folder=r"G:\ILA2024\ILA2024_dicom\test_folder"
maxdicom=find_max(test_folder)
# 文件夹路径
filepath = maxdicom
# 获取分辨率和层数
resolution, num_layers = get_resolution_and_layers(filepath)

# 批量处理

In [4]:
def dic2nii_sitk(path_read, path_save):
    reader = sitk.ImageSeriesReader()
    seriesIDs = reader.GetGDCMSeriesIDs(path_read)
    N = len(seriesIDs)
    lens = np.zeros([N])
    for i in range(N):
        dicom_names = reader.GetGDCMSeriesFileNames(path_read, seriesIDs[i])
        lens[i] = len(dicom_names)
    N_MAX = np.argmax(lens)
    dicom_names = reader.GetGDCMSeriesFileNames(path_read, seriesIDs[N_MAX])
    reader.SetFileNames(dicom_names)
    image = reader.Execute()
    sitk.WriteImage(image, path_save)



In [5]:
source_folder = r"G:\ILA2024\nii_test"
unzip_files = os.listdir(source_folder)
output_folder=r"G:\ILA2024\nii_test"
i=1
for unzip_file in unzip_files:
    print("--------------------" + str(i) + "------------------")
    i=i+1
    zip_file_path = os.path.join(source_folder, unzip_file)#每个病人
    maxfile_path=find_max(zip_file_path)
    #get_resolution_and_layers(maxfile_path)
    #print(maxfile_path)
    name=unzip_file+".nii.gz"
    
    outpath=os.path.join(output_folder,name)
    #print(outpath)
    
    dic2nii_sitk(maxfile_path,outpath)

--------------------1------------------


In [31]:
import os
import nibabel as nib
import pandas as pd


folder_path = r"G:\ILA2024\nii"
dataframe = get_resolution_and_layers(folder_path)

# 将 DataFrame 写入 CSV 文件
csv_file = "output.csv"
dataframe.to_csv(csv_file, index=False)
print(f"Data has been written to {csv_file}")


Data has been written to output.csv


# 针对带有时序信息的

In [ ]:

#如果是时序的
source_folder = r"G:\ILA2024\ILA2024_dicom"
unzip_files = os.listdir(source_folder)
output_folder=r"G:\ILA2024\nii"

for unzip_file in unzip_files:
    zip_file_path = os.path.join(source_folder, unzip_file)
    filetimes= os.listdir(zip_file_path)
    for filetime in filetimes:  #需要多一层时序层
        file_path=os.path.join(zip_file_path,filetime)
        #file_path=os.path.join(file_path,"ST0")
        print(file_path)
        maxfile_path=find_max(file_path)
        name=unzip_file+"_"+filetime+".nii.gz"
        print(maxfile_path)
        outpath=os.path.join(output_folder,name)
        
        dic2nii_sitk(file_path,outpath)

# 文件夹里有._开头的文件  通过os.walk遍历删除

In [8]:
import os

def remove_files_starting_with_dotunderscore(folder_path):
    # 列出文件夹中的所有文件和子文件夹
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            # 如果文件名以“._”开头，则删除它
            if file.startswith("._"):
                file_path = os.path.join(root, file)
                os.remove(file_path)

# 文件夹路径
folder_path = r"G:\ILA2024\ILA2024_dicom"

# 删除所有以“._”开头的文件
remove_files_starting_with_dotunderscore(folder_path)
